# Pre-match pipeline with tagged trials set

using the deepseek-r1:14b to tagged the patient and the trials

### install ollama and deepseek

ollama

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

deepseek-r1:14b

In [ ]:
!ollama pull deepseek-r1:14b

### load data

In [1]:
import pandas as pd
from fsspec.asyn import private
from sqlalchemy import false

# patient_df = pd.read_csv('./data/input/Prematch OncoPATH_20250303-07.csv')
patient_file = 'OncoPATH Prematch 20250630-0704'
trial_file = '06 NCCS_CT dd 10May2025'

patient_df = pd.read_excel(f'./data/input/patient/{patient_file}.xlsx')
trials_df = pd.read_csv(f'./data/input/trials/{trial_file}.csv')

trials_df.columns, patient_df.columns

(Index(['Trial Name', 'Site', 'Stage of cancer', 'Title of Study', 'Trial ID',
        'Unnamed: 5', 'Study Description', 'Inclusion Criteria',
        'Exclusion Criteria', 'Study Status', 'No. of patients accrued',
        'Target number', 'Phase', 'Targeting mutation'],
       dtype='object'),
 Index(['Patient_No', 'Primary', 'Stage', 'Clinical Note'], dtype='object'))

#### clean trials name

In [2]:
clear_name = []
for i, name in enumerate(trials_df['Trial Name']):
    # print(i,name)
    name = name.replace('\n', ' ').replace('\t', ' ')
    print(i,name)
    clear_name.append(name)
trials_df['Trial Name'] = clear_name
trials_df.to_csv(f'./data/input/trials/{trial_file}.csv', index=False)

0 TAPISTRY
1 BAYER 21820
2 EBC-129-01
3 NVL-520-01
4 PRT3789-01 
5 NVL-655-01
6 PAUF-I
7 CAAA603D12101
8 PMV-586-101
9 ZWI-ZW191-01
10 ALE.P02.01
11 SGNDV-001
12 BT8009-230
13 LaPemERLA
14 ELEVATE
15 INAVO122
16 TROPION-Breast04
17 CO44657
18 J2J-MC-JZLH / EMBER-4
19 MK-2870-012
20 ASPEN-06
21 D9800C00001 / CLARITY-PanTumour01
22 CA209-7R9
23 AHCC09
24 AHCC12
25 BO42777 (HORIZON01)
26 BI 1479-0008 (BEAMION LUNG-02)
27 HMBD-001-103
28 D5087C00001 SAFFRON
29 20230016 DeLLphi-306
30 BO45217 Krascendo 1
31 TAS6417-301 
32 LUNAR-2 EF-44 
33 20230153
34 V940-009
35 IOV-LUN-202
36 M25-274
37 GSK 213823
38 MK2140-006
39 POLAR
40 R1979-HM-2299  OLYMPIA-4
41 CA073-1020 GOLSEEK-1
42 M22-128
43 SG-AAA-II-01 / CAAA601A0SG01T
44 LDOXIRI-PDAC-01
45 CAAA617D12302
46 MK-5684-003
47 MK5684-004
48 CAAA817A12201
49 AMG 20230005
50 XL092-304
51 APL10101 SPARTA
52 TPX-0005-01 TRIDENT-1
53 NCCS-2018-Phase2-PRL3
54 STCC-01
55 MK6482-015
56 TORL123-002


### summarize the patient profile

the summarize model

In [3]:
from ollama import chat
from ollama import ChatResponse


def Ask_LLM(model, role, query):
    response: ChatResponse = chat(model=model,
                                  messages=[
                                      {'role': 'system', 'content': role},
                                      {'role': 'user', 'content': query},
                                  ],
                                  options={
                                      'temperature': 0  # very conservative (good for coding and correct syntax)
                                  })

    return response.message.content

summarize profile

de-identify process

In [4]:
from transformers import pipeline

pipe = pipeline("token-classification", model="StanfordAIMI/stanford-deidentifier-base", device="cuda")

In [5]:

def aggregate_masks(deindentify_marks):
    if len(deindentify_marks) == 0:
        return deindentify_marks
    new_deindentify_marks = []
    # print(deindentify_marks)
    new_deindentify_marks.append(deindentify_marks[0])
    for mark in deindentify_marks[1:]:
        if (mark['index'] - new_deindentify_marks[-1]['index'] == 1) and (
                mark['entity'] == new_deindentify_marks[-1]['entity']):
            new_deindentify_marks[-1]['end'] = mark['end']
            new_deindentify_marks[-1]['index'] = mark['index']
        else:
            new_deindentify_marks.append(mark)
    # print(new_deindentify_marks)
    return new_deindentify_marks


def deidentify(sentence, deindentify_marks):
    sentence = list(sentence)
    last_end = 0
    marked_sentence = ''
    aggregate_deindentify_marks = aggregate_masks(deindentify_marks)
    for mark in aggregate_deindentify_marks:
        marked_sentence += ''.join(sentence[last_end:mark['start']]) + '<' + mark['entity'] + '>'
        last_end = mark['end']
    if last_end < len(sentence):
        marked_sentence += ''.join(sentence[last_end:])
    return ''.join(marked_sentence)

test de-id

In [6]:
# summarized_patients = []
# de_identify_patients = []
# for patient, in zip(patient_df['Clinical Note']):
#     de_indentify_result = pipe(patient)
#     # print(de_indentify_result)
#     de_identify_patients.append(deidentify(patient, de_indentify_result))
#     print(de_identify_patients[-1])

summarize patient data

In [7]:
llm_query_role = 'you are an experienced oncology physician'
patient_llm_query_prompt = 'I have a patient which profile is below: {patient_profile}, can you tell the cancer site, cancer stage from it with no more information as the following structure: {structure}, Do not add preceding text before the abstraction, and do not add notes or commentary afterwards. This will not be used for clinical care, so do not write any disclaimers or cautionary notes.'
Ner_response_structure = {
    "diagnosis": 'cancer site',
    "stage": 'cancer stage'
}

In [8]:
summarized_patients = []
de_identify_patients = []
for patient in patient_df['Clinical Note']:
    de_indentify_result = pipe(patient)
    de_identify_patients.append(deidentify(patient, de_indentify_result))
    # print(deidentify(patient, de_indentify_result))
    summarized_patients.append(Ask_LLM(model='deepseek-r1:14b',
                                       role=llm_query_role,
                                       query=patient_llm_query_prompt.format(patient_profile=patient,
                                                                             structure=Ner_response_structure)))

patient_df['Clinical Note'] = de_identify_patients

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


~~summarize trials data~~  now the site and stage are given in the file

In [9]:
# llm_query_role = 'you are an experienced oncology physician'
# trial_llm_query_prompt = '''I have a trials information below: {trial_info}, can you tell the cancer site, cancer stage the trial requiring with no more information as the following structure: {structure}, if there are more than one sub trials, you can show as following structure {more_structure}. Use only double quotes in response. Just answering once using the structure. Do not add preceding text before the abstraction, and do not add notes or commentary afterwards. This will not be used for clinical care, so do not write any disclaimers or cautionary notes.'''
# Ner_response_structure = {
#     "diagnosis": 'cancer site',
#     "stage": 'cancer stage'
# }
# multi_structure = {
#     "1": {"diagnosis": "cancer site", "stage": "cancer stage"},
#     "2": {"diagnosis": "cancer site", "stage": "cancer stage"},
#     "3": {"diagnosis": "cancer site", "stage": "cancer stage"},
# }

In [10]:
# summarized_trials = []
# for title, study, inclusion in zip(trials_df['Title of Study'], trials_df['Study Description'],
#                                    trials_df['Inclusion Criteria']):
#     summarized_trials.append(Ask_LLM(model='deepseek-r1:14b',
#                                      role=llm_query_role,
#                                      query=trial_llm_query_prompt.format(trial_info=title + study + inclusion,
#                                                                          structure=Ner_response_structure,
#                                                                          more_structure=multi_structure)))

In [11]:
# for s in summarized_trials:
#     print(s)
#
# trials_df['site_stage_from_deepseek'] = summarized_trials

# if not os.path.exists('./data/output/example/'):
#     os.makedirs('./data/output/example/')
# trials_df.to_csv('./data/output/example/06 NCCS_CT dd 10May2025_deepseek.csv')

prepare patient data, get the NER result

In [12]:
import json


class Patient():
    def __init__(self, cancer_site, cancer_stage, patient_number):
        self.site = cancer_site
        self.stage = cancer_stage
        self.score = dict()
        self.mapped_site = []
        self.profile = ''
        self.patient_number = patient_number

    def add_profile(self, profile):
        self.profile = profile

    def mapping_site(self, mapped_site):
        self.mapped_site = mapped_site


summarized_patient_json = []
patient_list = []

for sp, profile, patient_number in zip(summarized_patients, patient_df['Clinical Note'], patient_df['Patient_No']):
    # print(sp)
    ind_left = sp.find('{')
    ind_right = sp.find('}')
    sp = sp[ind_left:ind_right + 1]
    # print(sp, profile)
    patient_json = json.loads(sp.replace("'", '"').replace("\\n", ""))
    summarized_patient_json.append(patient_json)
    patient = Patient(cancer_site=[patient_json['diagnosis']], cancer_stage=[patient_json['stage']], patient_number=patient_number)
    patient.add_profile(profile)
    patient_list.append(patient)

full_patient_profile_template = '''the patient is having {cancer_site}, which is {cancer_stage}, patient's known gene mutations are {gene}, known prior treatments are {treatments}. other related information are {related}'''
patient_site_stage_template = '''the patient is having {cancer_site}, which is {cancer_stage}'''

prepare trial data

if have old trials info, load and analysis

In [13]:
# summarized_trials_df = pd.read_csv('./data/output/example/06 NCCS_CT dd 10May2025_deepseek.csv')
# summarized_trials_df.columns

In [14]:
import json
import re


def find_all(string, sub):
    start = 0
    pos = []
    while True:
        start = string.find(sub, start)
        if start == -1:
            return pos
        pos.append(start)
        start += len(sub)


class Trial:
    def __init__(self, name, title, patient, phase):
        self.name = name
        self.title = title
        self.required_patients = patient
        self.phase = phase

    def add_required_patient(self, required_patient):
        self.required_patients.append(required_patient)


# summarized_trials_json = []
trials_list = []
phase_one_trial_list, normal_trial_list = [], []
for name, title, status, phase, tagged_site, tagged_stage in zip(trials_df['Trial Name'],
                                                                 trials_df['Title of Study'],
                                                                 trials_df['Study Status'],
                                                                 trials_df['Phase'],
                                                                 trials_df['Site'],
                                                                 trials_df['Stage of cancer']):

    if 'Open' not in status:
        t = Trial(name=name, title=title, patient=[], phase=phase)
        trials_list.append(t)
        continue
    # st = site_stage[0]
    # for st in summarized_trials:
    # i = st.find('</think>')
    # st = st[i + len('</think>'):]
    # print(st)
    # ind_left = find_all(st, '{')
    # ind_right = find_all(st, '}')
    # print(ind_left, ind_right)
    t = Trial(name=name, title=title, patient=[], phase=phase)
    # print('required: ', len(t.required_patients))
    t_site = [item.strip() for item in tagged_site.split(';') if item.strip()]
    # print(t_site)
    for site in t_site:
        tagged_patient_data = {
            'diagnosis': site,
            'stage': tagged_stage,
        }
        tagged_patient = Patient(cancer_site=[tagged_patient_data['diagnosis']],
                                 cancer_stage=[tagged_patient_data['stage']], patient_number = -1)
        t.add_required_patient(tagged_patient)

    if phase == '1' or phase == 1:
        phase_one_trial_list.append(t)
    else:
        normal_trial_list.append(t)
    trials_list.append(t)

    # for p in t.required_patients:
        # print(p.site, end='; ')

    # print(st)
    # summarized_trials_json.append(json.loads(st.replace("'", '"').replace("\\n", "")))

trials_site_stage_template = '''the trial is asking for patient who has {cancer_site}, which is {cancer_stage}'''

In [15]:
# for t in phase_one_trial_list:
#     print(len(t.required_patients))
# print(len(phase_one_trial_list), len(normal_trial_list))

### Two way to approach prematch here
- I am considering using code to process, based on the word in the tag then can build a dictionary
- or we can just transfer the tags and the patient cancer site and stage into vectors to get similarity

embedding model

In [16]:
from sentence_transformers import SentenceTransformer
import torch
from torch.nn import functional as F

embedding_model = SentenceTransformer('ksg-dfci/TrialSpace', trust_remote_code=True)


def embedding_out_tensor(sentence):
    with torch.no_grad():
        return embedding_model.encode(sentence, convert_to_tensor=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

#### code based:
1. match the patient to trial based on the tags
2. vectorize them(title, study description and inclusion criteria) and rank the prematch list by similarity score

##### do mapping

load mapping file

In [17]:
mapping_df = pd.read_csv('./data/output/Oncotree_list.csv')
# mapping_df.columns

1. separate the cancer site result into words
2. match the Code, if any matched, must be the one
3. match the Name, find the one that highest number of words matches into and return the Main Type as a result

In [18]:
def separate_into_words(sentence):
    start, end = 0, 0
    words = []
    for i, c in enumerate(sentence):
        end = i
        if i == len(sentence) - 1 and start != end:
            words.append(sentence[start:])
        if c.isalpha() or c.isdigit():
            continue
        else:
            if start == end:
                if i < len(sentence) - 1:
                    start, end = i + 1, i + 1
                else:
                    break
            else:
                words.append(sentence[start:end])
                if i < len(sentence) - 1:
                    start, end = i + 1, i + 1
                else:
                    break
    return words


'''
    I'm thinking if there are ways to do match just based on the cancer sites words, so separate the response into words, and find which cancer name contains most the words.
    if nothing matched, thought can just return 'cancer' as result
'''


def mapping_to_maintype(cancer_site):
    cancer_words = separate_into_words(cancer_site)
    mapped_cancer_type = []
    # print(mapped_cancer_type)
    max_matched_word_count = 0
    for code, name, maintype in zip(mapping_df['Code'], mapping_df['Name'], mapping_df['Maintype']):
        word_count = 0
        # print(cancer_words, 'match to', name, mapped_cancer_type)
        '''matched code means 100% match'''
        for word in cancer_words:
            if word.upper() == code.upper():
                mapped_cancer_type = [maintype]
                max_matched_word_count = len(cancer_words)

        '''count the match words number'''
        for word in cancer_words:
            if word.upper() in name.upper():
                word_count += 1
        if word_count == 0:
            # print('continue')
            continue
        '''higher means better match'''
        if word_count > max_matched_word_count:
            mapped_cancer_type = [maintype]
            max_matched_word_count = word_count
        '''keep the equal match'''
        if word_count == max_matched_word_count:
            if maintype not in mapped_cancer_type:
                # print(mapped_cancer_type)
                mapped_cancer_type.append(maintype)

    if len(mapped_cancer_type) == 0:
        mapped_cancer_type = ['Cancer']
    return mapped_cancer_type


'''mapping the patient and the trials'''
patient_origin_site, patient_mapped_site = [], []
for i, patient in enumerate(patient_list):
    mapped_site = []
    for site in patient.site:
        # print(i, site)
        mapped_site += mapping_to_maintype(site)
        # print(mapped_site)
    patient_list[i].mapping_site(mapped_site)
    patient_origin_site.append(patient.site)
    patient_mapped_site.append(mapped_site)

patient_df['origin site'] = patient_origin_site
patient_df['mapped site'] = patient_mapped_site

# if not os.path.exists('./data/output/prematch_exam/'):
#     os.makedirs('./data/output/prematch_exam/')
patient_df.to_csv(f'./data/output/prematch_exam/{patient_file}_mapped.csv', index=False)

for i, trial in enumerate(phase_one_trial_list):
    for p, patient in enumerate(phase_one_trial_list[i].required_patients):
        mapped_site = []
        for site in patient.site:
            # print(i, site)
            mapped_site += mapping_to_maintype(site)
            # print(mapped_site)
        phase_one_trial_list[i].required_patients[p].mapping_site(mapped_site)

for i, trial in enumerate(normal_trial_list):
    for p, patient in enumerate(normal_trial_list[i].required_patients):
        mapped_site = []
        for site in patient.site:
            # print(i, site)
            mapped_site += mapping_to_maintype(site)
            # print(mapped_site)
        normal_trial_list[i].required_patients[p].mapping_site(mapped_site)

trials_origin_site, trials_mapped_site = [], []
for i, trial in enumerate(trials_list):
    trial_patient_origin_site, trial_patient_mapped_site = [], []
    for p, patient in enumerate(trials_list[i].required_patients):
        mapped_site = []
        for site in patient.site:
            mapped_site += mapping_to_maintype(site)
        trials_list[i].required_patients[p].mapping_site(mapped_site)
        trial_patient_origin_site += patient.site
        trial_patient_mapped_site += mapped_site
    trials_origin_site.append(trial_patient_origin_site)
    trials_mapped_site.append(trial_patient_mapped_site)

trials_df['origin site'] = trials_origin_site
trials_df['mapped site'] = trials_mapped_site
# if not os.path.exists('./data/output/example/'):
#     os.makedirs('./data/output/example/')
trials_df.to_csv(f'./data/output/example/{trial_file}_deepseek_mapped.csv', index=False)

In [19]:
# mapping_to_maintype('solid tumor'), mapping_to_maintype('Solid tumor'), mapping_to_maintype('Solid Tumor')

In [20]:
# mapped_site = []
# for site in patient_list[0].site:
#     # print(site)
#     mapped_site += mapping_to_maintype(site)
#     # print(mapped_site)
# patient_list[0].mapping_site(mapped_site)
# print(patient_list[0].mapped_site, len(patient_list[0].mapped_site))

do match

In [21]:
llm_query_role = 'you are an experienced oncology physician'
patient_gene_mutation_llm_query_prompt = '''I have a patient which profile is below: {patient_profile}, tell the gene mutation the patient has from it. if it doesn't have, just return NaN. Do not add preceding text before the abstraction, and do not add notes or commentary afterwards. This will not be used for clinical care, so do not write any disclaimers or cautionary notes.'''
# Ner_response_structure = {
#     "diagnosis": 'cancer site',
#     "stage": 'cancer stage'
# }
# summarized_patients = []
# for patient in zip(patient_df['Clinical Note']):
#     summarized_patients.append(Ask_LLM(model='deepseek-r1:14b',
#                                        role=llm_query_role,
#                                        query=patient_llm_query_prompt.format(patient_profile=patient,
#                                                                              structure=Ner_response_structure)))

In [22]:
import os

def get_top(tup, number):
    key = []
    tup = sorted(tup, key=lambda x: x[0], reverse=True)[:number]
    for t in tup:
        key.append(t[1])
    return key


def range_trials(tup):
    key = []
    tup = sorted(tup, key=lambda x: x[0], reverse=True)
    for t in tup:
        key.append(str(t[1]))
    return key


def prematch_trials(trial_list, patient):
    pending_trials = []
    # print(len(pending_trials))
    for trial in tqdm(trial_list):
        patient_mapped_site = patient.mapped_site
        match_plag = False
        for required_patient in trial.required_patients:
            for pmappedsite in patient_mapped_site:
                if pmappedsite in required_patient.mapped_site:
                    # print(pmappedsite, required_patient.mapped_site)
                    match_plag = True
                    break
        if match_plag:
            # print('patient info: ', patient_mapped_site, patient.stage)
            trials_mapped_site, trials_stage = [], []
            for required_patient in trial.required_patients:
                trials_mapped_site += required_patient.mapped_site
                trials_stage += required_patient.stage
            # print('trials info: ', trials_mapped_site, trials_stage)
            pending_trials.append(trial)
    return pending_trials


from tqdm import tqdm

prematched_trial_for_patient = dict()
patient_id, patient_profiles, ctp = [], [], []
for patient in patient_list:
    patient_number = patient.patient_number
    print('patient No.', patient_number)
    '''code match the trials'''
    # separate the trial into phase_one and normal, do the match separately
    pending_trials = []

    pending_phase_one_trials = prematch_trials(trial_list=phase_one_trial_list, patient=patient)
    pending_normal_trials = prematch_trials(trial_list=normal_trial_list, patient=patient)

    # if the trials more than 10 or no gene mutation, just use the normal trials.
    gene_mutation_response = Ask_LLM(model='deepseek-r1:14b',
                                     role=llm_query_role,
                                     query=patient_gene_mutation_llm_query_prompt.format(patient_profile=patient))
    i = gene_mutation_response.find('</think>')
    gene_mutation = gene_mutation_response[i + len('</think>'):]

    # pending_trials = pending_normal_trials

    if 'nan' in gene_mutation.lower():
        pending_trials = pending_normal_trials + pending_phase_one_trials
    else:
        if len(pending_normal_trials) + len(pending_phase_one_trials) > 10:
            pending_trials = pending_normal_trials
        else:
            pending_trials = pending_normal_trials + pending_phase_one_trials

    # if len(pending_normal_trials) + len(pending_phase_one_trials) > 10:
    #     pending_trials = pending_normal_trials
    # else:
    #     if 'NaN' in gene_mutation:
    #         pending_trials = pending_normal_trials
    #     else:
    #         pending_trials = pending_normal_trials + pending_phase_one_trials

    '''get score'''
    score_list = []
    patient_sentence = patient_site_stage_template.format(cancer_site=patient.site,
                                                          cancer_stage=patient.stage)
    print('matched trials number:', len(pending_trials))
    for trial in tqdm(pending_trials):
        trial_required_sentence = []
        # print(len(trial.required_patients))
        for required_patient in trial.required_patients:
            trial_required_sentence.append(trials_site_stage_template.format(cancer_site=required_patient.site,
                                                                             cancer_stage=required_patient.stage))
        score = max(F.cosine_similarity(embedding_out_tensor(patient_sentence),
                                        embedding_out_tensor(trial_required_sentence)).cpu().numpy())
        score_list.append((score, trial.name))

    prematched_trial_name = range_trials(score_list)
    # print(prematched_trial_name)

    prematched_trial_name_str = ','.join(prematched_trial_name)

    patient_id.append('Patient' + str(patient_number))
    patient_profiles.append(patient.profile)
    ctp.append(prematched_trial_name_str)

    this_patient_dataframe = pd.DataFrame()
    this_patient_dataframe['no.'] = [0]
    this_patient_dataframe['Patient ID'] = ['Patient' + str(patient_number)]
    this_patient_dataframe['Patient Profile'] = [patient.profile]
    this_patient_dataframe['Trial Name'] = [prematched_trial_name_str]
    per_patient_save_path = f'./data/output/prematch/{patient_file}/'
    if not os.path.exists(per_patient_save_path):
        os.makedirs(per_patient_save_path)
    this_patient_dataframe.to_csv(per_patient_save_path + f'{patient_file} patient {patient_number} prematch result' + '.csv')

prematched_trial_for_patient['Patient ID'] = patient_id
prematched_trial_for_patient['Patient Profile'] = patient_profiles
prematched_trial_for_patient['Trial Name'] = ctp
prematched_trial_for_patient_df = pd.DataFrame(prematched_trial_for_patient)
prematched_trial_for_patient_df.to_csv(f'./data/output/prematch/{patient_file}.csv',
                                       index=False)

patient No. 0


100%|██████████| 46/46 [00:00<00:00, 354014.65it/s]


matched trials number: 0


0it [00:00, ?it/s]


patient No. 1


...

100%|██████████| 47/47 [00:07<00:00,  6.09it/s]


patient No. 33


100%|██████████| 46/46 [00:00<00:00, 219247.71it/s]


matched trials number: 8


100%|██████████| 8/8 [00:05<00:00,  1.58it/s]
